In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor, Lambda, Compose
%matplotlib
import matplotlib.pyplot as plt

C:\Users\root\anaconda3\envs\torch-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using matplotlib backend: <object object at 0x0000020D7B4AC930>


In [2]:
#Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

#Download the test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

classes = (
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
)

%matplotlib inline
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img/2 + 0.5
    npimg = img.numpy()
    if one_channel: 
        plt.plot(npimg)
    else:
        ptl.imshow(np.transpose(npimage, (1,2,0)))
    

In [4]:
#Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

#Define model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = x.view(-1, 16*4*4)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

model = Net().to(device)
print(model)


Using cuda device
Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU(inplace=True)
)


In [5]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [6]:
from torch.utils.tensorboard import SummaryWriter
writter = SummaryWriter("runs/fashion_mnist")

## Writing to Tensorboard

In [7]:
#get some random training images
data_iter = iter(train_dataloader)
images, labels = data_iter.next()

#create grid of images
img_grid = make_grid(images)

#show images
#matplotlib_imshow(img_grid, one_channel=True)

#write to tensorboard
writter.add_image('four_fasion_mnist_images', img_grid)


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #compute the prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        
        #Backpropagtion
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
images.shape

torch.Size([64, 1, 28, 28])

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n -------------------------------")
    train(train_dataloader, model, loss, optimizer)
    test(test_dataloader, model)
print("Done")

Epoch 1
 -------------------------------
loss: 2.292957 [    0/60000]
loss: 2.311231 [ 6400/60000]
loss: 2.299067 [12800/60000]
loss: 2.302268 [19200/60000]
loss: 2.289737 [25600/60000]
loss: 2.278692 [32000/60000]
loss: 2.253475 [38400/60000]
loss: 2.155120 [44800/60000]
loss: 1.694135 [51200/60000]
loss: 1.337099 [57600/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 0.017975 

Epoch 2
 -------------------------------
loss: 1.022878 [    0/60000]
loss: 1.039346 [ 6400/60000]
loss: 0.914827 [12800/60000]
loss: 0.928122 [19200/60000]
loss: 0.887062 [25600/60000]
loss: 0.730862 [32000/60000]
loss: 0.897100 [38400/60000]
loss: 0.739965 [44800/60000]
loss: 0.844458 [51200/60000]
loss: 0.681556 [57600/60000]
Test Error: 
 Accuracy: 70.5%, Avg loss: 0.012103 

Epoch 3
 -------------------------------
loss: 0.717838 [    0/60000]
loss: 0.757296 [ 6400/60000]
loss: 0.776201 [12800/60000]
loss: 0.597284 [19200/60000]
loss: 0.877646 [25600/60000]
loss: 0.863646 [32000/60000]
loss: 0.600088 [384

In [14]:
images = images.to(device)
writter.add_graph(model, images)
writter.close()

In [15]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
%reload_ext tensorboard